NO ES CODIGO TERMINADO. Lo dejo por si quiero comparar con algo o sacar ideas pero no esta chequeado.

### FUNCTIONAL IMPLEMENTATION 
Using torch.func

Parece ser que usar torch.func no sirve tanto para este contexto porque la paralelizacion de vmap no se lleva bien con collectar data de distintos envs ; esta pensado mas para parelizar tensor operations ( Si vendria bien para MAML en el supervised setting)

In [ ]:
from torch.func import functional_call, grad ,vmap

In [ ]:
def collect_data_from_env(params ,env,num_steps):
    #episode_return=0
    episodes_lengths=[]
    episodes_returns=[]

    #instantiate a buffer to save the data in
    buffer= Data_buffer(num_steps=num_steps ,env=env ,device=device)
    
    #get an initial state from the agent 
    #next_obs = torch.tensor(env.reset()[0],dtype=torch.float32).to(device)
    next_obs=env.reset()[0] #!!!
    done = torch.zeros(1).to(device)

    for step in range(0, num_steps):

        #prepare for new step: next_obs is the new step obs , the done flag which indicates wether the episode finished in the
        #last step becomes this episodes prev_done.
        obs, prev_done = next_obs, done

        obs=minigrid_preprocess_obs(obs).to(device) #!!! ---
        # get actionA action predictions and state value estimates
        with torch.no_grad():
            action, logprob, _, value = functional_call(agent,params,obs.unsqueeze(0))  #policy.get_action_and_value(obs.unsqueeze(0))

        #execute the action and get environment response.
        next_obs, reward, terminated, truncated, info = env.step(action.cpu().numpy())  #!!! - action.squeeze(0).cpu().numpy()
        done= torch.max(torch.tensor([terminated,truncated],dtype=torch.float32))#int(terminated or truncated)

        #preprocess and store data that you get from a step in a standard RL algo
        reward= torch.as_tensor(reward,dtype=torch.float32).to(device)

        buffer.store_step_data(step_index=step, obs=obs, act=action.long(), reward=reward, val=value
                                                        , logp=logprob,prev_done=prev_done)


        #prepare for next step
        #next_obs, done =torch.as_tensor(next_obs,dtype=torch.float32).to(device) ,torch.as_tensor(done,dtype=torch.float32).to(device)
        done = torch.as_tensor(done,dtype=torch.float32).to(device)

        #global_step += 1 
        #episode_step_num+=1 ## just for metaworld
        #episode_return+= reward
        
        #deal with the case where the episode ends for  the environment
        if done:
            #reset environment
            next_obs = env.reset()[0] #!! - next_obs = torch.Tensor(env.reset()[0]).to(device)
            
            assert 'episode' in info , 'problem with recordeEpisodeStatistics wrapper'
            episodes_returns.append(info['episode']['r'][0])
            episodes_lengths.append(info['episode']['l'][0])
            #if len(episodes_lengths)%10==0:
            #if global_step>print_info_factor*5000:
            #    print(f"global_step={global_step}, episodic_return={info['episode']['r'][0]} ,episodic_length={info['episode']['l'][0]} ")
            #    print_info_factor+=1
            
        
    # bootstrap value if not done
    with torch.no_grad():
        #get state value estimate of the new state
        processed_next_obs=minigrid_preprocess_obs(next_obs).to(device)
        _, _ , _, next_value = functional_call(agent,params,processed_next_obs.unsqueeze(0))
        #next_value = agent.get_value(processed_next_obs.unsqueeze(0))
        #next_value = policy.get_value(next_obs)
        
        #calculate the advantages and to go returns for each state visited in the data. They get stored in the buffer automatically
        buffer.calculate_returns_and_advantages(done,next_value,gae=config.il_gae,gamma=config.il_gamma,gae_lambda=config.il_gae_lambda)

    #calculate some metrics for future logging
    #print(np.array(episodes_returns).mean() ,np.array(episodes_lengths).mean())
    mean_episode_return= np.array(episodes_returns).mean()

    return buffer , mean_episode_return




def policy_loss(params,buffer,loss_type='vanilla'):
    '''Compute a pseudo loss from collected data such that the gradient of the pseudo loss
    coincides with the gradient of the true RL objective

    Args:
        buffer (_type_): _description_
        policy (_type_): _description_
    '''
    if loss_type== 'vanilla':
        #obs, act, adv, logp_old = buffer.observations.reshape((-1,) + env.observation_space['image'].shape), buffer.actions.reshape((-1,) + env.action_space.shape), data['adv'], data['logp']
        obs, act, adv, logp_old = buffer.observations , buffer.actions ,buffer.advantages, buffer.logprob_actions

        #print(buffer.observations.shape , buffer.observations.reshape((-1,) + env.observation_space['image'].shape).shape ,buffer.logprob_actions.shape, buffer.logprob_actions.reshape(-1).shape)

        # Policy loss
        _ , newlogprob, entropy, newvalue = functional_call(agent,params,(obs, act) )
        #_, newlogprob, entropy, newvalue = agent.get_action_and_value(obs, act)
        normalize_advantage=True
        if normalize_advantage==True:
            adv = (adv - adv.mean()) / (adv.std() + 1e-8)

        pg_loss = -(newlogprob * adv).mean()

        # Useful extra info
        with torch.no_grad():
            approx_kl = (logp_old - newlogprob).mean().item()

        #value loss
        v_loss = 0.5 * ((newvalue - buffer.returns) ** 2).mean()

        il_pseudo_policy_loss = pg_loss  + v_loss * config.il_valuef_coef
            

        return il_pseudo_policy_loss 
    
    #if loss_type=='ppo':

def policy_loss_for_adaptation(params,buffer,loss_type='vanilla'):
    return policy_loss(params,buffer,loss_type='vanilla')

def policy_loss_for_meta_update(params,buffer,loss_type='vanilla'):
    return policy_loss(params,buffer,loss_type='vanilla')

In [ ]:
def adaptation_inner_loop(base_params ,env ,logger=None):
    
    
    # collect data with base policy for doing the inner loop adaptation

    buffer, mean_episode_return= collect_data_from_env(params=base_params ,env=env, num_steps=config.env_steps_to_estimate_loss)
    if logger:
        logger.base_policy_episode_returns.append(mean_episode_return)
    
    # Compute gradient of the tasks's policy objective function (with data collected with the base policy)
    # with respect to the base policy parameters
    grads = grad(policy_loss_for_adaptation,argnums=0, has_aux=False)(base_params ,il_buffer)

    updated_params = OrderedDict()
    for (name, param), grad in zip(base_params.items(), grads):
        updated_params[name] = param - config.il_lr * grad
    

    # collect data with adapted policy for the outer loop update
    buffer,mean_episode_return =collect_data_from_env(params=updated_params ,env=env, num_steps=config.env_steps_to_estimate_loss)
    if logger:
        logger.adapted_policies_episode_returns.append(mean_episode_return)
    # Compute gradient of the tasks's policy objective function (with data collected with the adapted policy- the policy objective evaluated at the updated policy)
    # with respect to the base policy parameters
    adapted_pg_loss, adapted_v_loss = policy_loss_for_meta_update(params=updated_params ,buffer=buffer)
    meta_pseudo_policy_loss = adapted_pg_loss  + adapted_v_loss * config.il_valuef_coef

    return meta_pseudo_policy_loss

In [ ]:
def outer_loop():

    for i in range(config.num_meta_updates):
        meta_loss=0
        envs=[sample_env(size=env_size ,max_steps=max_episode_length) for _ in range(config.num_tasks_per_meta_update)]
        for env in envs:
            meta_loss+= adaptation_inner_loop(agent_model=agent, env=env, logger=logger)
            grads = grad(adaptation_inner_loop,argnums=0, has_aux=False)(agent.parameters() ,il_buffer)

        #grads = torch.autograd.grad(meta_loss,
        #                    agent.parameters())#,retain_graph=True)
            
        meta_optimizer.zero_grad()
        meta_loss.backward()
        meta_optimizer.step()
        
        print(f'completed meta update {i}')

In [ ]:
def outer_loop():

    vmap(grad(adaptation_inner_loop)) 

Tests

In [ ]:
def func1(a):
    env=sample_env(8,100)
    obs=func(env)
    return obs
def func(env):
    obs=env.reset()[0]
    obs=minigrid_preprocess_obs(obs)
    random_action_index = torch.randint(0, 6, (1,)).numpy()
    next_obs, reward, terminated, truncated, info = env.step(random_action_index)
    next_obs=minigrid_preprocess_obs(next_obs)
    return next_obs

envs=[sample_env(8,100) for i in range(8)]
a=torch.randn(8,10)
results=vmap(func1,randomness="different")(a)